# Coursera Capstone - Week 3 - Exercise 3 - Clustering


Create dataframe with Toronto neighborhoods data:

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']
# instantiate the dataframe
postalCodes = pd.DataFrame(columns=column_names)

soup = BeautifulSoup(page.text, 'html.parser')
soup.prettify()

trs = soup.findAll('tr')

for tr in trs:
    tds = tr.findAll('td')
    if len(tds) == 3:
        col1= tds[0].contents[0]
        col2 = tds[1].findAll("a", attrs={'title':True})
        if len(col2) > 0:
            col2=col2[0].text
            col3 = tds[2].findAll("a", attrs={'title': True})
            if len(col3) == 0:
                col3 = col2
            else:
                col3=col3[0].text
            postalCodes = postalCodes.append({'PostalCode': col1, 'Borough': col2, 'Neighborhood':col3}, ignore_index= True)

# print Postal Code Unique count to compare to final dataframe shape
print('Postal Code Unique count ', len(postalCodes['PostalCode'].unique()))
            
df = postalCodes.groupby(['PostalCode', 'Borough']).Neighborhood
postalCodes = df.agg(dict(Neighborhood=lambda x: list(x), number='count'))
postalCodes.reset_index(inplace=True)
postalCodes.drop('number', 1, inplace=True)

geoDf = pd.read_csv('https://cocl.us/Geospatial_data')
geoDf.rename(columns={'Postal Code': 'PostalCode'}, inplace= True)
postalCodes = pd.merge(postalCodes, geoDf, on='PostalCode')
postalCodes.head(5)

Postal Code Unique count  100


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Scarborough, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


Use only rows with the word 'Toronto' in Borough column

In [5]:
toronto_data = postalCodes[postalCodes['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head(14)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,[The Beaches],43.676357,-79.293031
1,M4K,East Toronto,"[East Toronto, Riverdale]",43.679557,-79.352188
2,M4L,East Toronto,"[East Toronto, India Bazaar]",43.668999,-79.315572
3,M4M,East Toronto,[East Toronto],43.659526,-79.340923
4,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790
5,M4P,Central Toronto,[Central Toronto],43.712751,-79.390197
6,M4R,Central Toronto,[Central Toronto],43.715383,-79.405678
7,M4S,Central Toronto,[Central Toronto],43.704324,-79.388790
8,M4T,Central Toronto,"[Moore Park, Central Toronto]",43.689574,-79.383160
9,M4V,Central Toronto,"[Deer Park, Central Toronto, Rathnelly, South ...",43.686412,-79.400049


Foursquare credentials

In [3]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100
radius=500

Function that explores neighborhoods


In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

Apply last function to Toronto's neighborhoods

In [7]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude'])


['The Beaches']
['East Toronto', 'Riverdale']
['East Toronto', 'India Bazaar']
['East Toronto']
['Lawrence Park']
['Central Toronto']
['Central Toronto']
['Central Toronto']
['Moore Park', 'Central Toronto']
['Deer Park', 'Central Toronto', 'Rathnelly', 'South Hill', 'Central Toronto']
['Rosedale']
['Cabbagetown', 'St. James Town']
['Church and Wellesley']
['Harbourfront', 'Regent Park']
['Ryerson', 'Downtown Toronto']
['St. James Town']
['Berczy Park']
['Downtown Toronto']
['Adelaide', 'King', 'Downtown Toronto']
['Downtown Toronto', 'Toronto Islands', 'Union Station']
['Design Exchange', 'Toronto Dominion Centre']
['Commerce Court', 'Downtown Toronto']
['Central Toronto']
['Forest Hill North', 'Central Toronto']
['The Annex', 'Central Toronto', 'Yorkville']
['Downtown Toronto', 'University of Toronto']
['Chinatown', 'Grange Park', 'Kensington Market']
['CN Tower', 'Downtown Toronto', 'Downtown Toronto', 'Downtown Toronto', 'King and Spadina', 'Railway Lands', 'South Niagara']
['Downt

Print Toronto's venues

In [8]:
print(toronto_venues.shape)
toronto_venues.head()

(1700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,[The Beaches],43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,[The Beaches],43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,[The Beaches],43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,"[East Toronto, Riverdale]",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
4,"[East Toronto, Riverdale]",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop


Encoding categories

In [9]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="",prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
# move neighborhood column to the first column
toronto_onehot = toronto_onehot[['Neighborhood'] + [c for c in toronto_onehot if c not in ['Neighborhood']]]
# change Neighborhood column type from list to tuple to groupby
toronto_onehot['Neighborhood'] = [tuple(x) for x in toronto_onehot['Neighborhood']] 

toronto_onehot.head()


,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"(The Beaches,)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(The Beaches,)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(The Beaches,)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(East Toronto, Riverdale)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(East Toronto, Riverdale)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
toronto_onehot.shape

(1700, 239)

Group categories by neighborhood

In [12]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"(Adelaide, King, Downtown Toronto)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000
1,"(Berczy Park,)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"(CN Tower, Downtown Toronto, Downtown Toronto,...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"(Cabbagetown, St. James Town)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,"(Central Toronto,)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.014493,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.014493
5,"(Chinatown, Grange Park, Kensington Market)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.050505,0.000000,0.050505,0.010101,0.000000,0.00,0.000000
6,"(Church and Wellesley,)",0.011494,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,...,0.000000,0.00000,0.00,0.000000,0.011494,0.011494,0.000000,0.011494,0.00,0.011494
7,"(Commerce Court, Downtown Toronto)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
8,"(Deer Park, Central Toronto, Rathnelly, South ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,...,0.000000,0.00000,0.00,0.000000,0.000000,0.071429,0.000000,0.000000,0.00,0.000000
9,"(Design Exchange, Toronto Dominion Centre)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000


Sort most common categories

In [14]:
# sort categories function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

Apply function to sort categories in dataframe

In [15]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()
# set

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"(Adelaide, King, Downtown Toronto)",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Restaurant,Bar,Bakery,Gym
1,"(Berczy Park,)",Coffee Shop,Cocktail Bar,Restaurant,Steakhouse,Café,Bakery,Italian Restaurant,Beer Bar,Pub,Seafood Restaurant
2,"(CN Tower, Downtown Toronto, Downtown Toronto,...",Airport Service,Airport Terminal,Airport Lounge,Plane,Sculpture Garden,Boutique,Harbor / Marina,Boat or Ferry,Airport Gate,Airport
3,"(Cabbagetown, St. James Town)",Coffee Shop,Restaurant,Pizza Place,Bakery,Italian Restaurant,Café,Pub,Chinese Restaurant,Park,Pharmacy
4,"(Central Toronto,)",Sandwich Place,Clothing Store,Coffee Shop,Dessert Shop,Pizza Place,Park,Diner,Burger Joint,Café,Sporting Goods Shop


Use k-means to cluster

In [16]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run kmeans clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 4, 4, 4, 4, 4, 0, 4], dtype=int32)

Add clustering labels to dataframe

In [17]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data
toronto_merged['Neighborhood'] = [tuple(x) for x in toronto_merged['Neighborhood']] 

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,"(The Beaches,)",43.676357,-79.293031,0,Coffee Shop,Pub,Yoga Studio,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"(East Toronto, Riverdale)",43.679557,-79.352188,4,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
2,M4L,East Toronto,"(East Toronto, India Bazaar)",43.668999,-79.315572,4,Sandwich Place,Gym,Brewery,Italian Restaurant,Steakhouse,Food & Drink Shop,Fish & Chips Shop,Light Rail Station,Fast Food Restaurant,Burger Joint
3,M4M,East Toronto,"(East Toronto,)",43.659526,-79.340923,4,Café,Coffee Shop,Brewery,Bakery,Light Rail Station,Park,Yoga Studio,Italian Restaurant,American Restaurant,Seafood Restaurant
4,M4N,Central Toronto,"(Lawrence Park,)",43.728020,-79.388790,2,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


Prepare to plot Toronto and clusters

In [18]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Plot Toronto and clusters

In [19]:
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],
toronto_merged['Longitude'], toronto_merged['Neighborhood'],
toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat, lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,
toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"(The Beaches,)",0,Coffee Shop,Pub,Yoga Studio,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
9,"(Deer Park, Central Toronto, Rathnelly, South ...",0,Coffee Shop,Pub,Convenience Store,Fried Chicken Joint,Bagel Shop,Sports Bar,Supermarket,American Restaurant,Pizza Place,Vietnamese Restaurant


Examine each cluster

In [20]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,
toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"(Moore Park, Central Toronto)",1,Playground,Park,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
10,"(Rosedale,)",1,Park,Trail,Playground,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio


In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,
toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"(Lawrence Park,)",2,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,
toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,"(Forest Hill North, Central Toronto)",3,Trail,Jewelry Store,Sushi Restaurant,Park,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,
toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"(East Toronto, Riverdale)",4,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
2,"(East Toronto, India Bazaar)",4,Sandwich Place,Gym,Brewery,Italian Restaurant,Steakhouse,Food & Drink Shop,Fish & Chips Shop,Light Rail Station,Fast Food Restaurant,Burger Joint
3,"(East Toronto,)",4,Café,Coffee Shop,Brewery,Bakery,Light Rail Station,Park,Yoga Studio,Italian Restaurant,American Restaurant,Seafood Restaurant
5,"(Central Toronto,)",4,Sandwich Place,Clothing Store,Coffee Shop,Dessert Shop,Pizza Place,Park,Diner,Burger Joint,Café,Sporting Goods Shop
6,"(Central Toronto,)",4,Sandwich Place,Clothing Store,Coffee Shop,Dessert Shop,Pizza Place,Park,Diner,Burger Joint,Café,Sporting Goods Shop
7,"(Central Toronto,)",4,Sandwich Place,Clothing Store,Coffee Shop,Dessert Shop,Pizza Place,Park,Diner,Burger Joint,Café,Sporting Goods Shop
11,"(Cabbagetown, St. James Town)",4,Coffee Shop,Restaurant,Pizza Place,Bakery,Italian Restaurant,Café,Pub,Chinese Restaurant,Park,Pharmacy
12,"(Church and Wellesley,)",4,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Pub,Men's Store,Mediterranean Restaurant,Fast Food Restaurant
13,"(Harbourfront, Regent Park)",4,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Theater,Pub,Mexican Restaurant,Restaurant,French Restaurant
14,"(Ryerson, Downtown Toronto)",4,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Italian Restaurant,Diner,Japanese Restaurant,Pizza Place
